# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  5.83it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.69it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.27it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.26it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.39it/s]



  0%|          | 0/23 [00:00<?, ?it/s]

...

 78%|███████▊  | 18/23 [00:05<00:01,  3.52it/s]

 83%|████████▎ | 19/23 [00:05<00:01,  3.68it/s]

 87%|████████▋ | 20/23 [00:05<00:00,  3.88it/s]

 96%|█████████▌| 22/23 [00:05<00:00,  4.36it/s]

100%|██████████| 23/23 [00:06<00:00,  3.78it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Dominiq and I am a passionate traveler and writer. I have been working as a travel writer for many years, traveling to over 40 countries and writing for numerous travel publications.
My passion for travel started when I was a child, growing up in a multicultural family. My parents were from different parts of the world, and I was exposed to different cultures and traditions from a young age. This sparked my curiosity and love for exploring new places and meeting new people.
Over the years, I have traveled to many parts of the world, from the bustling cities of Asia to the ancient ruins of Europe, and from the vibrant markets of Africa
Prompt: The president of the United States is
Generated text:  a man, a 45-year-old man named Joe Biden. He is the 46th president of the United States and is a Democrat. He was born on November 20, 1942, in Scranton, Pennsylvania, and grew up in Claymont, Delaware. Biden attended the University of Delaware and Sy

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Il

 Ilana

 Ilana and

 Ilana and I

...

 now, and it's closer than you think. From personal assistants to self-driving cars, AI is changing the world. In this book, leading AI researcher and entrepreneur, Dr. Andrew Ng, explores the possibilities and challenges of this technology. He shares his insights on how AI is transforming industries, from healthcare to education, and what this means for the future of work.
Through his own experiences as a pioneer in the field of AI, Dr. Ng shares stories of breakthroughs and setbacks, successes and failures. He explains the key concepts and technologies that are driving the AI revolution, including deep learning, natural language processing,

 now, and it's closer than you think. From personal assistants to self-driving cars, AI is changing the world. In this book, leading AI researcher and entrepreneur, Dr. Andrew Ng, explores the possibilities and challenges of this technology. He shares his insights on how AI is transforming industries, from healthcare to education, and what this means for the future of work.
Through his own experiences as a pioneer in the field of AI, Dr. Ng shares stories of breakthroughs and setbacks, successes and failures. He explains the key concepts and technologies that are driving the AI revolution, including deep learning, natural language processing, and

 now, and it's closer than you think. From personal assistants to self-driving cars, AI is changing the world. In this book, leading AI researcher and entrepreneur, Dr. Andrew Ng, explores the possibilities and challenges of this technology. He shares his insights on how AI is transforming industries, from healthcare to education, and what this means for the future of work.
Through his own experiences as a pioneer in the field of AI, Dr. Ng shares stories of breakthroughs and setbacks, successes and failures. He explains the key concepts and technologies that are driving the AI revolution, including deep learning, natural language processing, and computer

 now, and it's closer than you think. From personal assistants to self-driving cars, AI is changing the world. In this book, leading AI researcher and entrepreneur, Dr. Andrew Ng, explores the possibilities and challenges of this technology. He shares his insights on how AI is transforming industries, from healthcare to education, and what this means for the future of work.
Through his own experiences as a pioneer in the field of AI, Dr. Ng shares stories of breakthroughs and setbacks, successes and failures. He explains the key concepts and technologies that are driving the AI revolution, including deep learning, natural language processing, and computer vision

 now, and it's closer than you think. From personal assistants to self-driving cars, AI is changing the world. In this book, leading AI researcher and entrepreneur, Dr. Andrew Ng, explores the possibilities and challenges of this technology. He shares his insights on how AI is transforming industries, from healthcare to education, and what this means for the future of work.
Through his own experiences as a pioneer in the field of AI, Dr. Ng shares stories of breakthroughs and setbacks, successes and failures. He explains the key concepts and technologies that are driving the AI revolution, including deep learning, natural language processing, and computer vision.

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Peter, and I'm a music enthusiast. I have a deep love for indie rock music, and I spend most of my free time listening to my favorite artists and discovering new ones.
I'm excited to share my passion with you through this blog, where I'll be reviewing and discussing the latest music releases, sharing my favorite artists and bands, and even providing some tips and advice for music lovers.
So, sit back, relax, and let's dive into the world of indie rock music together!

**Recent Posts**

*   [Review: Arctic Monkeys - The Car](/review-the-car)
*   [Discovering New Music

Prompt: The capital of France is
Generated text:  known for its rich history, art, fashion, and cuisine. From the famous Eiffel Tower to the Louvre Museum, Paris is a city that has something to offer everyone. Here are some of the top things to do in Paris, in no particular order:
1. Visit the Eiffel Tower: The iconic Eiffel Tower is a must-visit attraction in Paris. You can tak

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Ashley

 Ashley and

 Ashley and I

 Ashley and I have

...

 all about collaboration
The convergence of multiple technologies is creating a powerful platform for AI, but the real future of AI lies in collaboration, not just between humans and machines, but between machines themselves.
The AI landscape is rapidly evolving, with the convergence of multiple technologies, such as cloud computing, edge computing, the Internet of Things (IoT), and blockchain, creating a powerful platform for AI. This platform is enabling AI to become more pervasive, efficient, and effective.
However, the real future of AI lies not just in the technology itself, but in how it can be used to collaborate with humans and other machines.

 all about collaboration
The convergence of multiple technologies is creating a powerful platform for AI, but the real future of AI lies in collaboration, not just between humans and machines, but between machines themselves.
The AI landscape is rapidly evolving, with the convergence of multiple technologies, such as cloud computing, edge computing, the Internet of Things (IoT), and blockchain, creating a powerful platform for AI. This platform is enabling AI to become more pervasive, efficient, and effective.
However, the real future of AI lies not just in the technology itself, but in how it can be used to collaborate with humans and other machines. Collaboration

 all about collaboration
The convergence of multiple technologies is creating a powerful platform for AI, but the real future of AI lies in collaboration, not just between humans and machines, but between machines themselves.
The AI landscape is rapidly evolving, with the convergence of multiple technologies, such as cloud computing, edge computing, the Internet of Things (IoT), and blockchain, creating a powerful platform for AI. This platform is enabling AI to become more pervasive, efficient, and effective.
However, the real future of AI lies not just in the technology itself, but in how it can be used to collaborate with humans and other machines. Collaboration is

 all about collaboration
The convergence of multiple technologies is creating a powerful platform for AI, but the real future of AI lies in collaboration, not just between humans and machines, but between machines themselves.
The AI landscape is rapidly evolving, with the convergence of multiple technologies, such as cloud computing, edge computing, the Internet of Things (IoT), and blockchain, creating a powerful platform for AI. This platform is enabling AI to become more pervasive, efficient, and effective.
However, the real future of AI lies not just in the technology itself, but in how it can be used to collaborate with humans and other machines. Collaboration is key

 all about collaboration
The convergence of multiple technologies is creating a powerful platform for AI, but the real future of AI lies in collaboration, not just between humans and machines, but between machines themselves.
The AI landscape is rapidly evolving, with the convergence of multiple technologies, such as cloud computing, edge computing, the Internet of Things (IoT), and blockchain, creating a powerful platform for AI. This platform is enabling AI to become more pervasive, efficient, and effective.
However, the real future of AI lies not just in the technology itself, but in how it can be used to collaborate with humans and other machines. Collaboration is key to

In [6]:
llm.shutdown()